In [32]:
!pip install elevenlabs
!pip install pygame sounddevice soundfile


DEPRECATION: Loading egg at /opt/anaconda3/lib/python3.11/site-packages/huggingface_hub-0.25.0-py3.8.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /opt/anaconda3/lib/python3.11/site-packages/huggingface_hub-0.25.0-py3.8.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
  Using cached pygame-2.6.1-cp311-cp311-macosx_11_0_arm64.whl.metadata (12 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 108.5 kB/s eta 0:00:0000:010:04
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.9/107.9 kB 306.5 kB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 337.5 kB/s eta 0:00:0000:0100:01


In [8]:
import requests
from datetime import datetime

API_KEY = 'koRm4O8n7QEoH6BopQSVPY9OtySOYiZD'

latitude = 26.2124
longitude = 78.1772  


url = f'https://api.tomorrow.io/v4/timelines?location={latitude},{longitude}&fields=temperature,precipitationProbability&timesteps=1h&units=metric&apikey={API_KEY}'

response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    daily_summary = {}

    
    for timestep in data['data']['timelines'][0]['intervals']:
        timestamp = timestep['startTime']
        temperature = timestep['values']['temperature']
        precipitation = timestep['values']['precipitationProbability']
        
        
        date = timestamp.split('T')[0]  
        
        
        if date not in daily_summary:
            daily_summary[date] = {
                'max_temp': temperature,
                'min_temp': temperature,
                'total_precipitation': precipitation
            }
        else:
            # Update max and min temperature
            if temperature > daily_summary[date]['max_temp']:
                daily_summary[date]['max_temp'] = temperature
            if temperature < daily_summary[date]['min_temp']:
                daily_summary[date]['min_temp'] = temperature
            
            # Accumulate precipitation
            daily_summary[date]['total_precipitation'] += precipitation



Date: 2024-10-18, Max Temperature: 24.69°C, Min Temperature: 22.54°C, Total Precipitation Probability: 0%
Date: 2024-10-19, Max Temperature: 33.71°C, Min Temperature: 21.75°C, Total Precipitation Probability: 0%
Date: 2024-10-20, Max Temperature: 33.29°C, Min Temperature: 22.03°C, Total Precipitation Probability: 0%
Date: 2024-10-21, Max Temperature: 33.59°C, Min Temperature: 21.41°C, Total Precipitation Probability: 0%
Date: 2024-10-22, Max Temperature: 34.6°C, Min Temperature: 24.4°C, Total Precipitation Probability: 0%
Date: 2024-10-23, Max Temperature: 33.64°C, Min Temperature: 24.08°C, Total Precipitation Probability: 0%


In [51]:
import os

from groq import Groq

client = Groq(
    api_key="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",
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": f"Given the dictionary {daily_summary} with dates, the max , min temp and precipitation of the date, write a proper statement in hindi to sumarize it, and this data is a future prediction of the next 5 daya for weather forecast for farmers,frame the statement in such ways only, please give only the hindi part, do not give the translation, just output the hindi part, do not give anything in English, the numbers should be specified only the highest and lowest of the whole 5 days and the precipiation amount",
        }
    ],
    model="llama3-8b-8192",
)

Hindi_Text = chat_completion.choices[0].message.content

In [52]:
print(Hindi_Text)

"पानी की नमी 0, और पांच दिनों के दौरान maximum 34.6 डिग्री सेल्सियस और minimum 21.41 डिग्री सेल्सियस का तापमान होगा."


In [56]:
import requests
import io
import pygame
import time
from pygame import mixer
import tempfile
import os
import sounddevice as sd
import soundfile as sf
import numpy as np

def text_to_speech_direct(text):
    """Convert Hindi text directly to speech"""
    API_KEY = "sk_960d5837e76361493bce03b2d3ad9f5dcde11abe20ca69b3"
    VOICE_ID = "21m00Tcm4TlvDq8ikWAM"
    
    url = f"https://api.elevenlabs.io/v1/text-to-speech/{VOICE_ID}"
    
    headers = {
        "Accept": "audio/mpeg",
        "Content-Type": "application/json",
        "xi-api-key": API_KEY
    }
    
    data = {
        "text": text,
        "model_id": "eleven_multilingual_v2",
        "voice_settings": {
            "stability": 1.0,
            "similarity_boost": 0.75
        }
    }
    
    try:
        print("Converting text to speech...")
        response = requests.post(url, json=data, headers=headers)
        
        if response.status_code == 200:
            print("✓ Conversion successful!")
            return response.content
        else:
            print(f"✗ Error: {response.status_code}")
            return None
            
    except Exception as e:
        print(f"✗ An error occurred: {str(e)}")
        return None

# Method 1: Using pygame (recommended for most cases)
def play_audio_pygame(audio_data):
    try:
        # Initialize pygame mixer
        pygame.mixer.init()
        
        # Create a temporary file to play the audio
        with tempfile.NamedTemporaryFile(delete=False, suffix='.mp3') as temp_audio:
            temp_audio.write(audio_data)
            temp_audio_path = temp_audio.name
        
        # Load and play the audio
        pygame.mixer.music.load(temp_audio_path)
        pygame.mixer.music.play()
        
        # Wait for the audio to finish
        while pygame.mixer.music.get_busy():
            time.sleep(0.1)
            
        # Clean up
        pygame.mixer.music.stop()
        pygame.mixer.quit()
        os.remove(temp_audio_path)
        
    except Exception as e:
        print(f"Error playing audio: {str(e)}")

# Method 2: Using sounddevice and soundfile (good for more control)
def play_audio_sounddevice(audio_data):
    try:
        # Save audio data to temporary file
        with tempfile.NamedTemporaryFile(delete=False, suffix='.mp3') as temp_audio:
            temp_audio.write(audio_data)
            temp_audio_path = temp_audio.name
        
        # Read audio file
        data, samplerate = sf.read(temp_audio_path)
        
        # Play audio
        sd.play(data, samplerate)
        sd.wait()  # Wait until audio is finished
        
        # Clean up
        os.remove(temp_audio_path)
        
    except Exception as e:
        print(f"Error playing audio: {str(e)}")

# Example usage
def main():
    # Test text
    hindi_text = Hindi_Text
    
    # Get audio data
    audio_data = text_to_speech_direct(hindi_text)
    
    if audio_data:
        print("\nPlaying audio using pygame...")
        play_audio_pygame(audio_data)
        
        # Uncomment to try the sounddevice method
        # print("\nPlaying audio using sounddevice...")
        # play_audio_sounddevice(audio_data)

if __name__ == "__main__":
    main()

Converting text to speech...
✓ Conversion successful!

Playing audio using pygame...
